# Setup Paths

In [1]:
# project root
root = "/home/timothy/Downloads/IC/"

In [ ]:
from os import environ

environ["nnUNet_raw_data_base"] = root + "nnUNet_raw_data_base"
environ["nnUNet_preprocessed"] = root + "nnUNet_preprocessed"
environ["RESULTS_FOLDER"] = root + "nnUNet_trained_models"


In [ ]:
# data source path
taskName = "Task000_HeadCircumference/"

base = root + "nnUNet_raw_data_base/nnUNet_raw_data/" + taskName
imagesTr = base + "imagesTr/"
imagesTs = base + "imagesTs/"
labelsTr = base + "labelsTr/"


# Train New Models

## Generate NIFTI from PNG

In [ ]:
# import utility packages
from nnunet.utilities.file_conversions import convert_2d_image_to_nifti
from os import listdir

In [ ]:
training = root + "raw_png_data/head_circumference/training/"
counter = 0
for i, f in enumerate(sorted(listdir(training))):
    if not 'Ann' in f:
        inImage = training + f
        inLabel = training + f[:-4] + "_Annotation.png"

        index = str(counter).zfill(3)
        counter += 1

        outImage = imagesTr + "headcircumference_" + index
        outLabel = labelsTr + "headcircumference_" + index

        convert_2d_image_to_nifti(inImage, outImage)
        convert_2d_image_to_nifti(inLabel, outLabel, is_seg=True, transform=lambda x: (x > 100).astype(int))

In [ ]:
testing = root + "raw_png_data/head_circumference/testing/"
for i, f in enumerate(listdir(testing)):
    inImage = testing + f

    index = str(i).zfill(3)

    outImage = imagesTs + "headcircumference_" + index

    convert_2d_image_to_nifti(inImage, outImage)

## Data Preprocessing and Setup

In [ ]:
# generate dataset json for training
from nnunet.dataset_conversion.utils import generate_dataset_json

generate_dataset_json(
    base + "dataset.json",
    imagesTr,
    imagesTs,
    ("Ultrasound",),
    labels={0: "background", 1: "head"},
    dataset_name="Head Circumference",
    license="hands off!",
)


In [ ]:
!nnUNet_plan_and_preprocess -t 000 -pl3d None --verify_dataset_integrity

## Model Training

In [ ]:
!nnUNet_train 2d nnUNetTrainerV2 Task000_HeadCircumference 0 --npz

In [ ]:
!nnUNet_train 2d nnUNetTrainerV2 Task000_HeadCircumference 1 --npz

In [ ]:
!nnUNet_train 2d nnUNetTrainerV2 Task000_HeadCircumference 2 --npz

In [ ]:
!nnUNet_train 2d nnUNetTrainerV2 Task000_HeadCircumference 3 --npz

In [ ]:
!nnUNet_train 2d nnUNetTrainerV2 Task000_HeadCircumference 4 --npz

In [ ]:
!nnUNet_predict -i /home/timothy/Downloads/IC/nnUNet_raw_data_base/nnUNet_raw_data/Task000_HeadCircumference/imagesTr -o out/trained_output/Task000 -t 0 -m 2d

# Prediction Postprocessing

In [2]:
# post processed image paths
out = "out/trained_output/Task000/"
mapped = out + "mapped/"
png = out + "png/"


In [ ]:
# map region values
from os import listdir
import SimpleITK as sitk

writer = sitk.ImageFileWriter()

for f in listdir(out):
    if f.endswith(".gz"):
        arr = sitk.GetArrayFromImage(sitk.ReadImage(out + f)) * 255
        img = sitk.GetImageFromArray(arr)

        writer.SetFileName(mapped + f)
        writer.Execute(img)


In [ ]:
# convert mapped nii.gz to png
from nnunet.utilities.file_conversions import convert_2d_segmentation_nifti_to_img

for f in listdir(mapped):
    convert_2d_segmentation_nifti_to_img(mapped + f, png + f[18:21] + "_HC.png")


# Evaluation

In [3]:
# metrics
diceSum = 0  # Dice Coefficient
hDistSum = 0  # Hausdorff Distance
sqrErr = 0  # RMSE value
circumferenceSum = 0 # circumferences
count = 0


In [4]:
# load image as array

import numpy as np
import SimpleITK as sitk


def loadImg(filename):
    arr = sitk.GetArrayFromImage(sitk.ReadImage(filename))

    coords = []
    X = []
    Y = []

    for y, row in enumerate(arr):
        for x, pxl in enumerate(row):
            if pxl > 100:
                coords.append((x, y))
                X.append(x)
                Y.append(y)

    return coords, np.array(X), np.array(Y)

In [5]:
# calculate dice coefficient

def getDiceCoefficient(X, Y):
    overlapCount = 0
    for point in X:
        if point in Y:
            overlapCount += 1

    return overlapCount * 2 / (len(X) + len(Y))

In [6]:
# calculate Hausdorff distance metric


def dist(xX, xY, yX, yY):
    return np.sqrt((xX - yX) ** 2 + (xY - yY) ** 2)


def getHausdorffDistance(X, Y):
    # loop relative to set X
    supXx = 0
    for xPt in X:
        infYx = float("inf")
        xX, xY = xPt
        for yPt in Y:
            yX, yY = yPt
            infYx = min(infYx, dist(xX, xY, yX, yY))

        supXx = max(supXx, infYx)

    return supXx


In [7]:
# estimate perimeter
# source: https://scipython.com/blog/direct-linear-least-squares-fitting-of-an-ellipse/


def fitEllipse(x, y):
    D1 = np.vstack([x**2, x * y, y**2]).T
    D2 = np.vstack([x, y, np.ones(len(x))]).T
    S1 = D1.T @ D1
    S2 = D1.T @ D2
    S3 = D2.T @ D2
    T = -np.linalg.inv(S3) @ S2.T
    M = S1 + S2 @ T
    C = np.array(((0, 0, 2), (0, -1, 0), (2, 0, 0)), dtype=float)
    M = np.linalg.inv(C) @ M
    eigval, eigvec = np.linalg.eig(M)
    con = 4 * eigvec[0] * eigvec[2] - eigvec[1] ** 2
    ak = eigvec[:, np.nonzero(con > 0)[0]]
    return np.concatenate((ak, T @ ak)).ravel()


def toStdCoeff(coeffs):
    a = coeffs[0]
    b = coeffs[1] / 2
    c = coeffs[2]
    d = coeffs[3] / 2
    f = coeffs[4] / 2
    g = coeffs[5]

    den = b**2 - a * c
    x0, y0 = (c * d - b * f) / den, (a * f - b * d) / den

    num = 2 * (a * f**2 + c * d**2 + g * b**2 - 2 * b * d * f - a * c * g)
    fac = np.sqrt((a - c) ** 2 + 4 * b**2)

    ap = np.sqrt(num / den / (fac - a - c))
    bp = np.sqrt(num / den / (-fac - a - c))

    width_gt_height = True
    if ap < bp:
        width_gt_height = False
        ap, bp = bp, ap

    if b == 0:
        phi = 0 if a < c else np.pi / 2
    else:
        phi = np.arctan((2.0 * b) / (a - c)) / 2
        if a > c:
            phi += np.pi / 2
    if not width_gt_height:
        phi += np.pi / 2
    phi = phi % np.pi

    return x0, y0, ap, bp, phi


def getApproxPerim(xIn, yIn):
    x0, y0, ap, bp, phi = toStdCoeff(fitEllipse(xIn, yIn))
    h3 = 3 * ((ap - bp) ** 2 / (ap + bp) ** 2)
    return np.pi * (ap + bp) * (1 + h3 / (10 + np.sqrt(4 - h3)))


In [8]:
# evaluate predictions

import csv

with open(
    root + "raw_png_data/head_circumference/training_set_pixel_size_and_HC.csv",
    newline="",
) as csvFile:
    spamreader = csv.reader(csvFile, delimiter=",", quotechar="|")
    for i, row in enumerate(spamreader):
        if i != 0 and i > 799:
            prediction = png + str(i - 1).zfill(3) + "_HC.png"
            target = (
                root
                + "raw_png_data/head_circumference/training/"
                + row[0][:-4]
                + "_Annotation.png"
            )
            scaleFactor = float(row[1])
            hcExact = float(row[2])

            pCoords, pX, pY = loadImg(prediction)
            tCoords, tX, tY = loadImg(target)

            sX = pX * scaleFactor
            sY = pY * scaleFactor
            try:
                print(i - 1)
                dice = getDiceCoefficient(pCoords, tCoords)
                hDist = getHausdorffDistance(pCoords, tCoords) * scaleFactor
                circumference = np.round_(getApproxPerim(sX, sY), 2)
                err = (hcExact - circumference) ** 2

                print(dice, hDist, err)
                diceSum += dice
                hDistSum += hDist
                sqrErr += err
                circumferenceSum += hcExact
                count += 1
                print(diceSum / count, hDistSum / count, np.sqrt(sqrErr / count))
                print()

            except:
                pass


799
0.08419417475728155 2.25657590592005 0.8835999999999957
0.08419417475728155 2.25657590592005 0.9399999999999977

800
0.4200851826944631 1.3815420949441006 0.1848999999999814
0.25213967872587234 1.8190590004320752 0.730924072664178

801
0.34076587493940863 2.4951698584373543 0.0015999999999993634
0.28168174413038444 2.044429286433835 0.5972436688655579

802
0.253210718249564 1.2643944475624118 1.166400000000027
0.2745639876601793 1.8494205767159793 0.7477466148368717

803
0.27475642161204605 1.987856865059923 5.107599999999959
0.2746024744505527 1.877107834384768 1.2119488438048829

804
0.29501267962806427 1.827992250741127 11.764900000000047
0.27800417531347127 1.8689219037774947 1.7846101348287067

805
0.34669943035853906 2.322588686588143 9.985599999999978
0.2878177831770524 1.9337314441790157 2.038718084623626

806
0.14158009180961584 3.1264268035443012 29.26810000000027
0.2695380717561228 2.0828183640996762 2.7009882450688365

807
0.22982142857142857 2.0977726264735446 14.13760

In [9]:
circumferenceSum / count

275.48175000000015